In [1]:
import pandas as pd
import random
from datetime import datetime, timedelta

# 1. Clientes
clientes_data = [
    {'cliente_id': 1, 'nombre_cliente': 'Cruz Verde'},
    {'cliente_id': 2, 'nombre_cliente': 'Embonor'},
    {'cliente_id': 3, 'nombre_cliente': 'Supermercados Rey'}
]
df_clientes = pd.DataFrame(clientes_data)

# 2. Categorias
categorias_data = [
    {'categoria_id': 101, 'nombre_categoria': 'Bebidas'},
    {'categoria_id': 102, 'nombre_categoria': 'Snacks'},
    {'categoria_id': 103, 'nombre_categoria': 'Farmacia'},
    {'categoria_id': 104, 'nombre_categoria': 'Cuidado Personal'},
    {'categoria_id': 105, 'nombre_categoria': 'Lácteos'}
]
df_categorias = pd.DataFrame(categorias_data)

# 3. Cliente_Categoria 
cliente_categoria_data = [
    {'cliente_id': 1, 'categoria_id': 103},
    {'cliente_id': 1, 'categoria_id': 104},
    {'cliente_id': 1, 'categoria_id': 102},
    {'cliente_id': 2, 'categoria_id': 101},
    {'cliente_id': 2, 'categoria_id': 102},
    {'cliente_id': 2, 'categoria_id': 103},
    {'cliente_id': 2, 'categoria_id': 104},
    {'cliente_id': 2, 'categoria_id': 105},
    {'cliente_id': 3, 'categoria_id': 101},
    {'cliente_id': 3, 'categoria_id': 105}
]
df_cliente_categoria = pd.DataFrame(cliente_categoria_data)

# 4. Auditorias (Ajustado a 10 ejemplos)
auditorias_data = []
tiendas = ['Tienda Norte', 'Tienda Sur', 'Tienda Centro', 'Tienda Este']
auditores = ['Ana Perez', 'Carlos Gomez', 'Luis Silva']

# Bucle ajustado para generar exactamente 10 registros
for i in range(1, 11):
    cliente_seleccionado = random.choice([1, 2, 3])
    auditorias_data.append({
        'auditoria_id': i,
        'fecha': (datetime.now() - timedelta(days=random.randint(0, 10))).strftime('%Y-%m-%d'),
        'tienda': random.choice(tiendas),
        'auditor_responsable': random.choice(auditores),
        'cliente_id': cliente_seleccionado
    })
df_auditorias = pd.DataFrame(auditorias_data)

# 5. FotosCargadas
fotos_data = []
foto_id_counter = 1

for _, auditoria in df_auditorias.iterrows():
    cat_esperadas = df_cliente_categoria[df_cliente_categoria['cliente_id'] == auditoria['cliente_id']]['categoria_id'].tolist()
    
    # Semilla de alerta: definimos aleatoriamente si esta auditoría será deficiente
    es_alerta = random.choice([True, False]) 
    
    # Fotos de inicio y fin (Control)
    fotos_a_cargar = 2 if not es_alerta else random.choice([0, 1])
    
    for _ in range(fotos_a_cargar):
        fotos_data.append({
            'foto_id': foto_id_counter,
            'auditoria_id': auditoria['auditoria_id'],
            'categoria_id': None, 
            'tipo': 'Control'
        })
        foto_id_counter += 1
        
    # Fotos por categoría
    for cat in cat_esperadas:
        if es_alerta and random.random() > 0.5: # Aumenté la probabilidad de que falten fotos para asegurar alertas en los 10 ejemplos
            continue
            
        fotos_data.append({
            'foto_id': foto_id_counter,
            'auditoria_id': auditoria['auditoria_id'],
            'categoria_id': cat,
            'tipo': 'Categoria'
        })
        foto_id_counter += 1

df_fotos_cargadas = pd.DataFrame(fotos_data)

print(f"¡Datos generados exitosamente! Total de auditorías: {len(df_auditorias)}")

¡Datos generados exitosamente! Total de auditorías: 10


In [2]:
# 1. Calcular cuántas categorías audita cada cliente
cat_por_cliente = df_cliente_categoria.groupby('cliente_id')['categoria_id'].count().reset_index()
cat_por_cliente.rename(columns={'categoria_id': 'total_categorias'}, inplace=True)

# 2. Calcular el mínimo de fotos esperadas (categorías + 2 obligatorias de control)
cat_por_cliente['fotos_esperadas'] = cat_por_cliente['total_categorias'] + 2

# 3. Enriquecer las auditorías con los nombres de clientes y las fotos esperadas
auditorias_info = df_auditorias.merge(df_clientes, on='cliente_id', how='left')
auditorias_esperadas = auditorias_info.merge(cat_por_cliente, on='cliente_id', how='left')

# 4. Contar cuántas fotos se cargaron realmente por cada auditoría
fotos_por_auditoria = df_fotos_cargadas.groupby('auditoria_id')['foto_id'].count().reset_index()
fotos_por_auditoria.rename(columns={'foto_id': 'fotos_cargadas'}, inplace=True)

# 5. Unir la expectativa (lo esperado) con la realidad (lo cargado)
validacion_df = auditorias_esperadas.merge(fotos_por_auditoria, on='auditoria_id', how='left')

# Si una auditoría no tiene ninguna foto cargada, el cruce arrojará NaN. Lo rellenamos con 0.
validacion_df['fotos_cargadas'] = validacion_df['fotos_cargadas'].fillna(0).astype(int)

# 6. Calcular la diferencia y filtrar las alertas
validacion_df['fotos_faltantes'] = validacion_df['fotos_esperadas'] - validacion_df['fotos_cargadas']

# Una alerta es cualquier auditoría donde falte al menos 1 foto
df_alertas = validacion_df[validacion_df['fotos_faltantes'] > 0].copy()

# 7. Formatear la tabla final de alertas según el requerimiento exacto
tabla_alertas_final = df_alertas[['auditoria_id', 'nombre_cliente', 'fotos_faltantes']].reset_index(drop=True)

print("--- Tabla de Alertas Generada ---")
print(tabla_alertas_final)

--- Tabla de Alertas Generada ---
   auditoria_id     nombre_cliente  fotos_faltantes
0             1  Supermercados Rey                3
1             3            Embonor                3
2             5            Embonor                4
3             7  Supermercados Rey                2
4             8         Cruz Verde                3
